In [1]:
import os
os.chdir('../../data')
%pwd

'/Users/chrissoria/Documents/Programming /Data-Science-Social-Justice-main/data'

In [2]:
import pandas as pd 

df = pd.read_csv('disability_30000_submissions.csv')

In [3]:
%who

df	 os	 pd	 


In [4]:
df.head()

,Unnamed: 0,id,created_utc,link_flair_text,author_flair_text,author,over_18,title,is_self,selftext,score,upvote_ratio,distinguished,num_comments
0,0,p8hcfj,1629505393,NaN,NaN,OrdanCoal,False,Anybody noticed the trap people set with your ...,True,"Like, if I'm crying because I'm in chronic pai...",34,0.92,NaN,53
1,1,p84kgg,1629463903,Rant,NaN,dusnood_,False,"Feeling guilty for spending with ""disability i...",True,I don’t have any regular bills as I still live...,32,0.94,NaN,65
2,2,p8z4a1,1629579321,Image,NaN,sushimewmew,False,My old braces vs. my new brace. I'm very happy...,False,NaN,17,0.96,NaN,43
3,3,p82mfs,1629455803,Discussion,NaN,ShenandoahValley,False,Only 30% of SSD applicants approved on first try,True,This came from a manager at the local SSA offi...,14,0.90,NaN,81
4,4,p8cdd2,1629488445,NaN,NaN,musicalearnightingal,False,Have you ever called someone out for treating ...,True,So one of my coworkers really hates me. I get ...,12,0.93,NaN,20


In [5]:
df.shape

(30339, 14)

In [6]:
list(df)

['Unnamed: 0',
 'id',
 'created_utc',
 'link_flair_text',
 'author_flair_text',
 'author',
 'over_18',
 'title',
 'is_self',
 'selftext',
 'score',
 'upvote_ratio',
 'distinguished',
 'num_comments']

In [7]:
df = df.loc[~df['selftext'].isin(['[removed]', '[deleted]' ]),:]
df = df.dropna(subset=['selftext']).reset_index()
df.index

RangeIndex(start=0, stop=14928, step=1)

The dataset goes from 30,339 to 14,928 after deleting all removed and deleted selftext inputs and Null values. This seems like a dramatic reduction. 

In [8]:
import spacy
# Load the English preprocessing pipeline
nlp = spacy.load('en_core_web_sm')

# Parse the first reddit post in the dataset
parsed_post = nlp(df.selftext[0])
print(parsed_post)

Like, if I'm crying because I'm in chronic pain, it's because I'm "too weak to be happy"

If I'm happy and smile, it means I couldn't possibly be in pain

The fucked up thing is, I can definitely be in pain and still be happy, but if I have to meet one more nasty person that I thought I would like, but turns out they're horrible Ableist, I might vomit. 

The people that know me know me as the person that is legitimately in pain **and** is still smiling, but then these people that I would have vouched for as neighbors, friends, and otherwise the kind of people you'd genuinely want to meet, suddenly turned out to be complete bigots. 

It's like coming to find out a dear member of your family is, like, **dangerously** racist, or that your Uncle doesn't see women as people. 

I think one of the worst things being disabled and, for some of us, debatably, perhaps *the* worst thing, is the faith you lose not just in life, but in humanity. 

Like, I was okay with being on a tiny spinning rock 

In [9]:
for idx, sentence in enumerate(parsed_post.sents):
    print(f'Sentence {idx + 1}')
    print(sentence)
    print('')

Sentence 1
Like, if I'm crying because I'm in chronic pain, it's because I'm "too weak to be happy"

If I'm happy and smile, it means I couldn't possibly be in pain



Sentence 2
The fucked up thing is, I can definitely be in pain and still be happy, but if I have to meet one more nasty person that I thought I would like, but turns out they're horrible Ableist, I might vomit.

Sentence 3




Sentence 4
The people that know me know me as the person that is legitimately in pain **and** is still smiling, but then these people that I would have vouched for as neighbors, friends, and otherwise the kind of people you'd genuinely want to meet, suddenly turned out to be complete bigots.

Sentence 5




Sentence 6
It's like coming to find out a dear member of your family is, like, **dangerously** racist, or that your Uncle doesn't see women as people.

Sentence 7




Sentence 8
I think one of the worst things being disabled and, for some of us, debatably, perhaps *the* worst thing, is the faith

This actually did a decent job of caputring "sentences." The two exceptions are Sentence 7 and Sentence 9, both blank. 

In [10]:
# Extract the first 15 items for the following properties of the parsed post

# The token text 
token_text = [token.orth_ for token in parsed_post][:15]   
# Part of speech 
token_pos = [token.pos_ for token in parsed_post][:15]   
# Lemma (or 'dictionary form')
token_lemma = [token.lemma_ for token in parsed_post][:15]
# Stop word? t/f
token_stop = [token.is_stop for token in parsed_post][:15]
# Puncutation? t/f
token_punct = [token.is_punct for token in parsed_post][:15]

# Make a dataframe with these items
pd.DataFrame(zip(token_text, token_pos, token_lemma, token_stop, token_punct),
             columns=['token_text', 'part_of_speech', 'token_lemma', 'token_stop', 'token_punct'])

,token_text,part_of_speech,token_lemma,token_stop,token_punct
0,Like,INTJ,like,False,False
1,",",PUNCT,",",False,True
2,if,SCONJ,if,True,False
3,I,PRON,I,True,False
4,'m,AUX,be,True,False
5,crying,VERB,cry,False,False
6,because,SCONJ,because,True,False
7,I,PRON,I,True,False
8,'m,AUX,be,True,False
9,in,ADP,in,True,False


It seems to be correctly identifying things.

In [11]:
def clean(token):
    """Helper function that specifies whether a token is:
        - punctuation
        - space
        - digit
    """
    return token.is_punct or token.is_space or token.is_digit

def line_read(df, text_col='selftext'):
    """Generator function to read in text from df and get rid of line breaks."""    
    for text in df[text_col]:
        yield text.replace('\n', '')

def preprocess(df, text_col='selftext', allowed_postags=['NOUN', 'ADJ']):
    """Preprocessing function to apply to a dataframe."""
    for parsed in nlp.pipe(line_read(df, text_col), batch_size=1000, disable=["tok2vec", "ner"]):
        # Gather lowercased, lemmatized tokens
        tokens = [token.lemma_.lower() if token.lemma_ != '-PRON-'
                  else token.lower_ 
                  for token in parsed if not clean(token)]
        # Remove specific lemmatizations, and words that are not nouns or adjectives
        tokens = [lemma
                  for lemma in tokens
                  if not lemma in ["'s",  "’s", "’"] and not lemma in allowed_postags]
        # Remove stop words
        tokens = [token for token in tokens if token not in spacy.lang.en.stop_words.STOP_WORDS]
        yield tokens

In [12]:
lemmas = [line for line in preprocess(df)]

In [13]:
%who

clean	 df	 idx	 lemmas	 line_read	 nlp	 os	 parsed_post	 pd	 
preprocess	 sentence	 spacy	 token_lemma	 token_pos	 token_punct	 token_stop	 token_text	 


In [14]:
lemmas[0]

['like',
 'crying',
 'chronic',
 'pain',
 'weak',
 'happy"if',
 'happy',
 'smile',
 'means',
 'possibly',
 'painthe',
 'fucked',
 'thing',
 'definitely',
 'pain',
 'happy',
 'meet',
 'nasty',
 'person',
 'thought',
 'like',
 'turns',
 'horrible',
 'ableist',
 'vomit',
 'people',
 'know',
 'know',
 'person',
 'legitimately',
 'pain',
 'smiling',
 'people',
 'vouched',
 'neighbors',
 'friends',
 'kind',
 'people',
 'genuinely',
 'want',
 'meet',
 'suddenly',
 'turned',
 'complete',
 'bigots',
 'like',
 'coming',
 'find',
 'dear',
 'member',
 'family',
 'like',
 'dangerously',
 'racist',
 'uncle',
 'women',
 'people',
 'think',
 'worst',
 'things',
 'disabled',
 'debatably',
 'worst',
 'thing',
 'faith',
 'lose',
 'life',
 'humanity',
 'like',
 'okay',
 'tiny',
 'spinning',
 'rock',
 'edge',
 'centerless',
 'void',
 'realize',
 'little',
 'company',
 'fellows',
 'care',
 'mean',
 "that's",
 'worse',
 'definitely',
 'worse']

In [15]:
parsed_post_test = nlp(df.selftext[10])
print(parsed_post_test)
lemmas[10]

Title, basically? An attorney advised me to find services/products that help me perform my part-time job, and that I’d be able to deduct their cost from my earnings for the process of determining eligibility.

I’ll examine my own life to see what I might be able to cite, but I’m at a loss and would love to hear examples of what other people have cited!


['title',
 'basically',
 'attorney',
 'advised',
 'find',
 'services',
 'products',
 'help',
 'perform',
 'time',
 'job',
 'able',
 'deduct',
 'cost',
 'earnings',
 'process',
 'determining',
 'eligibility',
 'i’ll',
 'examine',
 'life',
 'able',
 'cite',
 'loss',
 'love',
 'hear',
 'examples',
 'people',
 'cited']

The lemmas here look different than the post

In [16]:
print(df.selftext[10])

Title, basically? An attorney advised me to find services/products that help me perform my part-time job, and that I’d be able to deduct their cost from my earnings for the process of determining eligibility.

I’ll examine my own life to see what I might be able to cite, but I’m at a loss and would love to hear examples of what other people have cited!


In [17]:
print(df.index)

RangeIndex(start=0, stop=14928, step=1)


In [18]:
from gensim.models.phrases import Phrases, Phraser

bigram = Phrases(lemmas, min_count=10, threshold=100)
trigram = Phrases(bigram[lemmas], min_count=10, threshold=50)  
bigram_phraser = Phraser(bigram)
trigram_phraser = Phraser(trigram)

# Form trigrams
trigrams = [trigram_phraser[bigram_phraser[doc]] for doc in lemmas]

In [19]:
trigrams_joined = [' '.join(trigram) for trigram in trigrams]
trigrams_joined[0]

'like crying chronic_pain weak happy"if happy smile means possibly painthe fucked thing definitely pain happy meet nasty person thought like turns horrible ableist vomit people know know person legitimately pain smiling people vouched neighbors friends kind people genuinely want meet suddenly turned complete bigots like coming find dear member family like dangerously racist uncle women people think worst things disabled debatably worst thing faith lose life humanity like okay tiny spinning rock edge centerless void realize little company fellows care mean that\'s worse definitely worse'

In [20]:
trigram_phraser["That", "was", "not", "a", "big", "deal"]

['That', 'was', 'not', 'a', 'big', 'deal']

In [21]:
len(bigram_phraser.phrasegrams.keys()) #423 Bigrams were identified in the dataset

423

In [22]:
list(bigram_phraser.phrasegrams.keys())[:10] #what is the 10 corresponding to? 

['thanks_advance',
 'muscular_dystrophy',
 'official_diagnosis',
 'onset_date',
 'bachelor_degree',
 'wheelchair_user',
 'muscle_weakness',
 'l4_l5',
 'spinal_cord',
 'handicap_parking']

In [23]:
[trigram for trigram in list(trigram_phraser.phrasegrams.keys()) if trigram.count('_') == 2]

['substantial_gainful_activity',
 'spinal_cord_injury',
 'major_depressive_disorder',
 'generalized_anxiety_disorder',
 'ambulatory_wheelchair_user',
 'borderline_personality_disorder',
 'fetal_alcohol_syndrome',
 'playing_video_games',
 'administrative_law_judge',
 'obsessive_compulsive_disorder',
 'social_security_administration',
 'parked_handicap_spot',
 'auditory_processing_disorder',
 'short_term_memory',
 'play_video_games',
 'traumatic_brain_injury',
 'assisted_living_facility',
 'greatly_appreciated_thank',
 '\u200d_♀_️',
 'mild_cerebral_palsy',
 'ehlers_danlos_syndrome',
 'walk_short_distances',
 'residual_functional_capacity',
 'degenerative_disc_disease',
 'brother_cerebral_palsy',
 'mixed_connective_tissue',
 'remotely_fullest_extent',
 'spinal_muscular_atrophy',
 'california_department_rehabilitation']

In [24]:
# Inserting next to selftext column
df.insert(loc=7, column='lemmas', value=trigrams_joined)
# Removing empty rows in lemmas
df = df[~df['lemmas'].isin([''])]

In [25]:
 df.head(3)

,index,Unnamed: 0,id,created_utc,link_flair_text,author_flair_text,author,lemmas,over_18,title,is_self,selftext,score,upvote_ratio,distinguished,num_comments
0,0,0,p8hcfj,1629505393,NaN,NaN,OrdanCoal,"like crying chronic_pain weak happy""if happy s...",False,Anybody noticed the trap people set with your ...,True,"Like, if I'm crying because I'm in chronic pai...",34,0.92,NaN,53
1,1,1,p84kgg,1629463903,Rant,NaN,dusnood_,regular bills live relatives pay small things ...,False,"Feeling guilty for spending with ""disability i...",True,I don’t have any regular bills as I still live...,32,0.94,NaN,65
2,3,3,p82mfs,1629455803,Discussion,NaN,ShenandoahValley,came manager local ssa office va spoke started...,False,Only 30% of SSD applicants approved on first try,True,This came from a manager at the local SSA offi...,14,0.90,NaN,81


In [26]:
# Save to new csv
df.to_csv('disability_sub_top_sm_lemmas.csv', index=False)

Why did it generate 30,000 entries instead of 16,000 like we had ended up with after filter nulls and na?

In [27]:
%who

Phraser	 Phrases	 bigram	 bigram_phraser	 clean	 df	 idx	 lemmas	 line_read	 
nlp	 os	 parsed_post	 parsed_post_test	 pd	 preprocess	 sentence	 spacy	 token_lemma	 
token_pos	 token_punct	 token_stop	 token_text	 trigram	 trigram_phraser	 trigrams	 trigrams_joined	 
